# Airport Network
Open-Source Data from https://ourairports.com/data dan https://openflights.org/data.php

In [ ]:
import pandas as pd
import numpy as np
import random
import itertools
import networkx as nx
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt
import time
from scipy.stats import gaussian_kde
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import Normalize, LinearSegmentedColormap
from pyvis.network import Network

# Load the CSV file for the airports
airports_df = pd.read_csv('airports.csv')

airports_df.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
def on_scroll(event): 
    # Zoom in or out 
    base_scale = 1.2 
    if event.button == 'up':  # Scroll up 
        scale_factor = 1 / base_scale 
    elif event.button == 'down':  # Scroll down 
        scale_factor = base_scale 
    else: 
        return 
 
    cur_xlim = ax.get_xlim() 
    cur_ylim = ax.get_ylim() 
    xdata = event.xdata  # Get mouse x position 
    ydata = event.ydata  # Get mouse y position 
 
    # Calculate new limits 
    new_width = (cur_xlim[1] - cur_xlim[0]) * scale_factor 
    new_height = (cur_ylim[1] - cur_ylim[0]) * scale_factor 
    relx = (cur_xlim[1] - xdata) / (cur_xlim[1] - cur_xlim[0]) 
    rely = (cur_ylim[1] - ydata) / (cur_ylim[1] - cur_ylim[0]) 
 
    ax.set_xlim([xdata - new_width * (1 - relx), xdata + new_width * relx]) 
    ax.set_ylim([ydata - new_height * (1 - rely), ydata + new_height * rely]) 
    ax.figure.canvas.draw() 
 
# Connect the event 
fig.canvas.mpl_connect('scroll_event', on_scroll)

In [ ]:
# Load the CSV file, replacing "NA" with "North America"
df2 = pd.read_csv('airports_big.csv', na_values=["NA"])

# Replace "NA" with "North America" after loading
df2.fillna("North America", inplace=True)

# Filter the data
filtered_df2 = df2[df2['type'].isin(['large_airport', 'medium_airport'])]

In [ ]:
# Filter df where either 'ICAO' or 'IATA' is in filtered_df2['ident']
filtered_df = airports_df[
    airports_df['ICAO'].isin(filtered_df2['ident']) | airports_df['IATA'].isin(filtered_df2['ident'])
]
# print(filtered_df2[filtered_df2['name'] == 'Los Angeles International Airport'])

# Merge the filtered DataFrame with the continent column from filtered_df2
filtered_df = filtered_df.merge(filtered_df2[['ident', 'continent']], left_on='ICAO', right_on='ident', how='left')
# print(filtered_df[filtered_df['name'] == 'Los Angeles International Airport'])

# Drop the 'ident' column from the merged DataFrame as it's not needed anymore
filtered_df = filtered_df.drop(columns=['ident'])

# Display the filtered DataFrame with the continent column
filtered_df.head()

In [ ]:
print(filtered_df.shape)

# Airports are the Nodes

In [ ]:
# Create a custom blue-to-red colormap
blue_to_red = LinearSegmentedColormap.from_list('BlueToRed', ['blue', 'lightgray', 'red'])

# Create a figure for the globe map
plt.figure(figsize=(12, 8))

# Set up the Basemap with a Robinson projection for a global view
m = Basemap(projection='robin', lon_0=0, resolution='c')  # Center at 0 longitude

# Draw map features
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='lightgray', lake_color='aqua')
m.drawmapboundary(fill_color='aqua')

# Convert latitude and longitude to map coordinates
x, y = m(filtered_df['lon'].values, filtered_df['lat'].values)

# Perform kernel density estimation (KDE) to calculate density
positions = np.vstack([x, y])
kde = gaussian_kde(positions)
density = kde(positions)

# Normalize density to enhance visibility
norm = Normalize(vmin=np.percentile(density, 5), vmax=density.max())  # Clip lower percentiles
density_normalized = norm(density)

# Plot the airport locations with the custom colormap
sc = m.scatter(x, y, s=5, c=density_normalized, cmap=blue_to_red, marker='o', alpha=1, label='Airports')

# Add a colorbar to show density levels
# cbar = plt.colorbar(sc, label='Point Density')
# cbar.ax.set_ylabel('Point Density', rotation=270, labelpad=15)

# Add a legend
plt.legend(loc='lower left')

# Add a title
plt.title("Global Airport Locations")

# Show the map
plt.show()

# These are all used routes (edges)

In [ ]:
# Load the CSV file
edges = pd.read_csv('routes.csv')

# Select only the routes and destination columns (3 and 5)
routes = edges[['source_airport_id', 'destination_airport_id']]

# This still shows all available routes between airports, heliports and others that are not accessible by the average person, so we need to filter it again
routes

In [ ]:
print(routes['source_airport_id'].dtype) 
print(routes['destination_airport_id'].dtype)

# Convert both 'source_airport_id' and 'destination_airport_id' to numeric, forcing non-numeric values to NaN
routes['source_airport_id'] = pd.to_numeric(routes['source_airport_id'], errors='coerce')
routes['destination_airport_id'] = pd.to_numeric(routes['destination_airport_id'], errors='coerce')

# Drop rows where either 'source_airport_id' or 'destination_airport_id' is NaN
routes = routes.dropna(subset=['source_airport_id', 'destination_airport_id'])

# Convert both columns to int64
routes['source_airport_id'] = routes['source_airport_id'].astype('int64')
routes['destination_airport_id'] = routes['destination_airport_id'].astype('int64')

# Verify the data types
print(routes[['source_airport_id', 'destination_airport_id']].dtypes)

routes

In [ ]:
available_routes = routes[
    routes['source_airport_id'].isin(filtered_df['id']) & 
    routes['destination_airport_id'].isin(filtered_df['id'])
]
# Create a 'reverse' pair column to track the reverse direction
available_routes['reverse_pair'] = list(zip(available_routes['destination_airport_id'], available_routes['source_airport_id']))

# Remove duplicates, including reverse pairs
unique_routes = []
seen_routes = set()

for index, row in available_routes.iterrows():
    pair = (row['source_airport_id'], row['destination_airport_id'])
    reverse_pair = (row['destination_airport_id'], row['source_airport_id'])

    # If neither the pair nor the reverse pair has been seen before, add it to the unique_routes list
    if pair not in seen_routes and reverse_pair not in seen_routes:
        unique_routes.append(row)
        seen_routes.add(pair)

# Convert the list back into a DataFrame
available_routes = pd.DataFrame(unique_routes)

# Drop the 'reverse_pair' column
available_routes = available_routes.drop(columns=['reverse_pair'])

# Display the filtered DataFrame
print(available_routes)


In [ ]:
# Assign a unique number to each continent code in the 'continent' column
labels, uniques = pd.factorize(filtered_df['continent'])
filtered_df['continent'] = labels + 1  # Assign unique numbers starting from 1

# Create a mapping dictionary
continent_mapping = {index + 1: value for index, value in enumerate(uniques)}

# Display the mapping
print(continent_mapping)


In [ ]:
# Display the updated DataFrame
filtered_df.head()

# NX

In [ ]:
# Create a new graph
G = nx.Graph()

# Add nodes from the 'id' column of filtered_df
# G.add_nodes_from(filtered_df['id'], group=row["continent"])

for index, row in filtered_df.iterrows():
    G.add_node(row["id"], group=row["continent"], name=row["name"])

# Add edges
edges = list(zip(available_routes['source_airport_id'], available_routes['destination_airport_id']))
G.add_edges_from(edges)

# Display basic information about the graph
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")


In [ ]:
nx.is_connected(G)

In [ ]:
nx.number_connected_components(G)

# Draws graph for all airports locations

In [ ]:
# Define the drawing function
def draw_graph(G, size, withLabelsBool):
    # Remove nodes with 0 degree
    G = G.copy()  # Work on a copy to avoid modifying the original graph
    nodes_to_remove = [n for n, degree in G.degree() if degree == 0]
    G.remove_nodes_from(nodes_to_remove)
    
    # Define a color map
    color_map = {1: '#f09494', 2: '#eebcbc', 3: '#72bbd0', 4: '#91f0a1', 5: '#629fff', 6: '#bcc2f2', 7: '#eebcbc'}
    
    # Assign colors to nodes based on their 'group' attribute
    node_color = []
    for n, d in G.nodes(data=True):
        group = d.get('group', None)  # Get 'group' attribute safely
        if group in color_map:
            node_color.append(color_map[group])
        else:
            node_color.append('#fff')  # Default color for missing or unexpected group values
    
    # Layout for nodes
    pos = nx.drawing.kamada_kawai_layout(G)
    # pos = nx.spring_layout(G)
    
    # Extract node labels (e.g., airport names)
    labels = {n: d['name'] for n, d in G.nodes(data=True)}
    
    # Create the plot
    plt.figure(figsize=size)
    nx.draw_networkx(G, pos=pos, node_color=node_color, edge_color='#FFDEA2', with_labels=withLabelsBool, labels=labels, font_size=8)
    # nx.draw_networkx(G, pos=pos, node_color=node_color, edge_color='#FFDEA2', with_labels=not withLabelsBool)
    plt.show()

# Call the drawing function with the desired figure size
draw_graph(G, (25, 25), withLabelsBool=False)

# Subgraph for each continents

## Oceanic

{1: 'OC', 2: 'North America', 3: 'EU', 4: 'AF', 5: 'SA', 6: 'AS', 7: 'AN'}

In [ ]:
# Define the drawing function
def draw_graph_oc(G, size):
    # Define a color map
    color_map = {1: '#f09494', 2: '#eebcbc', 3: '#72bbd0', 4: '#91f0a1', 5: '#629fff', 6: '#bcc2f2', 7: '#eebcbc'}
    
    # Filter the graph to include only nodes where 'group' == 2
    nodes_group_2 = [n for n, d in G.nodes(data=True) if d.get('group') == 1]
    
    # Create a subgraph with only these nodes
    subgraph = G.subgraph(nodes_group_2)
    
    # Assign colors to nodes based on their 'group' attribute (just for the subgraph)
    node_color = []
    for n, d in subgraph.nodes(data=True):
        group = d.get('group', None)  # Get 'group' attribute safely
        if group in color_map:
            node_color.append(color_map[group])
        else:
            node_color.append('#fff')  # Default color for missing or unexpected group values
    
    # Layout for nodes
    pos = nx.drawing.kamada_kawai_layout(subgraph)
    
    # Extract node labels (airport names)
    labels = {n: d['name'] for n, d in subgraph.nodes(data=True)}
    
    # Create the plot
    plt.figure(figsize=size)
    nx.draw_networkx(subgraph, pos=pos, node_color=node_color, edge_color='#FFDEA2', labels=labels, font_size=8)
    plt.show()

# Call the drawing function with the desired figure size
draw_graph_oc(G, (30, 30))

## North America

In [ ]:
# Define the drawing function
def draw_graph_na(G, size):
    # Define a color map
    color_map = {1: '#f09494', 2: '#eebcbc', 3: '#72bbd0', 4: '#91f0a1', 5: '#629fff', 6: '#bcc2f2', 7: '#eebcbc'}
    
    # Filter the graph to include only nodes where 'group' == 2
    nodes_group_2 = [n for n, d in G.nodes(data=True) if d.get('group') == 2]
    
    # Create a subgraph with only these nodes
    subgraph = G.subgraph(nodes_group_2)
    
    # Assign colors to nodes based on their 'group' attribute (just for the subgraph)
    node_color = []
    for n, d in subgraph.nodes(data=True):
        group = d.get('group', None)  # Get 'group' attribute safely
        if group in color_map:
            node_color.append(color_map[group])
        else:
            node_color.append('#fff')  # Default color for missing or unexpected group values
    
    # Layout for nodes
    pos = nx.drawing.kamada_kawai_layout(subgraph)
    
    # Extract node labels (airport names)
    labels = {n: d['name'] for n, d in subgraph.nodes(data=True)}
    
    # Create the plot
    plt.figure(figsize=size)
    nx.draw_networkx(subgraph, pos=pos, node_color=node_color, edge_color='#FFDEA2', labels=labels, font_size=8)
    plt.show()

# Call the drawing function with the desired figure size
draw_graph_na(G, (30, 30))

## EU

In [ ]:
# Define the drawing function
def draw_graph_eu(G, size):
    # Define a color map
    color_map = {1: '#f09494', 2: '#eebcbc', 3: '#72bbd0', 4: '#91f0a1', 5: '#629fff', 6: '#bcc2f2', 7: '#eebcbc'}
    
    # Filter the graph to include only nodes where 'group' == 2
    nodes_group_3 = [n for n, d in G.nodes(data=True) if d.get('group') == 3]
    
    # Create a subgraph with only these nodes
    subgraph = G.subgraph(nodes_group_3)
    
    # Assign colors to nodes based on their 'group' attribute (just for the subgraph)
    node_color = []
    for n, d in subgraph.nodes(data=True):
        group = d.get('group', None)  # Get 'group' attribute safely
        if group in color_map:
            node_color.append(color_map[group])
        else:
            node_color.append('#fff')  # Default color for missing or unexpected group values
    
    # Layout for nodes
    pos = nx.drawing.kamada_kawai_layout(subgraph)
    
    # Extract node labels (airport names)
    labels = {n: d['name'] for n, d in subgraph.nodes(data=True)}
    
    # Create the plot
    plt.figure(figsize=size)
    nx.draw_networkx(subgraph, pos=pos, node_color=node_color, edge_color='#FFDEA2', labels=labels, font_size=8)
    plt.show()

# Call the drawing function with the desired figure size
draw_graph_eu(G, (30, 30))

In [ ]:
# Initialize pyvis network
net = Network(notebook=True, height="800px", width="100%")

# Add nodes and edges from NetworkX graph
net.from_nx(G)

# Customize appearance (optional)
net.force_atlas_2based()  # Force-directed layout

# Show the network
net.show("graph.html")

In [ ]:
from community import community_louvain

partition = community_louvain.best_partition(G)

num_communities = len(set(partition.values()))
print(f"Number of communities: {num_communities}")

# Subgraph Biggest Community 

In [ ]:
# Define a function to find the largest community and display it with labels as 'name'
def display_largest_community(G, size=(25, 25), withLabelsBool=True):
    # Compute communities using the Louvain method
    partition = community_louvain.best_partition(G)
    
    # Group nodes by community
    community_groups = {}
    for node, community_id in partition.items():
        if community_id not in community_groups:
            community_groups[community_id] = []
        community_groups[community_id].append(node)
    
    # Find the largest community
    largest_community_id = max(community_groups, key=lambda k: len(community_groups[k]))
    largest_community_nodes = community_groups[largest_community_id]
    
    # Extract the subgraph of the largest community
    largest_community_subgraph = G.subgraph(largest_community_nodes)
    
    # Extract node labels (based on 'name' attribute)
    labels = {n: G.nodes[n].get('name', str(n)) for n in largest_community_nodes}
    
    # Draw the largest community subgraph
    pos = nx.spring_layout(largest_community_subgraph)  # Layout for the largest community
    plt.figure(figsize=size)
    nx.draw(
        largest_community_subgraph,
        pos,
        with_labels=withLabelsBool,
        labels=labels,
        node_color='#72bbd0',
        edge_color='#FFDEA2',
        font_size=10,
        node_size=500,
    )
    plt.title(f"Largest Community (Size: {len(largest_community_nodes)})")
    plt.show()

display_largest_community(G)

In [ ]:
# Calculate the degree of each node in the graph
degrees = [G.degree(n) for n in G.nodes()]

# Calculate the average degree
average_degree = np.mean(degrees)
print(f"Average degree: {average_degree}")

# Degree distribution: Frequency of each degree
degree_count = {}
for degree in degrees:
    if degree not in degree_count:
        degree_count[degree] = 1
    else:
        degree_count[degree] += 1

# Print the degree distribution
print("Degree Distribution:")
for degree, count in sorted(degree_count.items()):
    print(f"Degree {degree}: {count} nodes")

# Plotting the degree distribution
plt.figure(figsize=(10, 6))
plt.bar(degree_count.keys(), degree_count.values(), color='#72bbd0')
# plt.xscale('log')
# plt.yscale('log')
plt.xlabel('Degree')
plt.ylabel('Number of nodes')
plt.title('Degree Distribution of the Graph')
plt.show()

### What if we remove the empty nodes?

In [ ]:
# Remove nodes with degree 0
nodes_to_remove = [n for n in G.nodes() if G.degree(n) == 0]
G.remove_nodes_from(nodes_to_remove)

# Recalculate degrees after removing nodes
degrees = [G.degree(n) for n in G.nodes()]

# Recalculate average degree
average_degree = np.mean(degrees)
print(f"Average degree after removing nodes with degree 0: {average_degree}")

# Recalculate degree distribution
degree_count = {}
for degree in degrees:
    if degree not in degree_count:
        degree_count[degree] = 1
    else:
        degree_count[degree] += 1

# Print the degree distribution
print("Degree Distribution after removing nodes with degree 0:")
for degree, count in sorted(degree_count.items()):
    print(f"Degree {degree}: {count} nodes")

# Plot the updated degree distribution
plt.figure(figsize=(10, 6))
plt.bar(degree_count.keys(), degree_count.values(), color='#72bbd0')
plt.xlabel('Degree')
plt.ylabel('Number of nodes')
plt.title('Updated Degree Distribution of the Graph')
plt.show()


# Max Degree

In [ ]:
# Find the airport (node) with the highest degree
max_degree_node = max(G.degree, key=lambda x: x[1])  # (node, degree) pair, sorting by degree
max_degree_airport_id = max_degree_node[0]  # The airport's ID with the highest degree
max_degree = max_degree_node[1]  # The highest degree value

# Find the corresponding airport name using the 'name' attribute
airport_name = G.nodes[max_degree_airport_id]['name']

print(f"The airport with the most connections (highest degree) is: {airport_name} with {max_degree} connections.")

## Subgraph of Amsterdam

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Define the drawing function
def draw_graph(G, size, withLabelsBool, center_name=None):
    # Remove nodes with 0 degree
    G = G.copy()  # Work on a copy to avoid modifying the original graph
    nodes_to_remove = [n for n, degree in G.degree() if degree == 0]
    G.remove_nodes_from(nodes_to_remove)
    
    # Define a color map
    color_map = {1: '#f09494', 2: '#eebcbc', 3: '#72bbd0', 4: '#91f0a1', 5: '#629fff', 6: '#bcc2f2', 7: '#eebcbc'}
    
    # Assign colors to nodes based on their 'group' attribute
    node_color = []
    for n, d in G.nodes(data=True):
        group = d.get('group', None)  # Get 'group' attribute safely
        if group in color_map:
            node_color.append(color_map[group])
        else:
            node_color.append('#fff')  # Default color for missing or unexpected group values
    
    # Find the node corresponding to the center name
    if center_name:
        center_node = None
        for n, d in G.nodes(data=True):
            if d.get('name') == center_name:
                center_node = n
                break
        if center_node:
            # Arrange the layout with the center node in the middle
            neighbors = list(G.neighbors(center_node))
            pos = {center_node: (0, 0)}  # Place the center node at (0, 0)
            
            # Arrange neighbors in a circle around the center
            num_neighbors = len(neighbors)
            for i, neighbor in enumerate(neighbors):
                angle = 2 * 3.14159 * i / num_neighbors  # Divide evenly around a circle
                pos[neighbor] = (1.5 * np.cos(angle), 1.5 * np.sin(angle))
            
            # Place other nodes randomly around
            for n in G.nodes():
                if n not in pos:
                    pos[n] = nx.drawing.spring_layout(G, center=center_node)[n]
        else:
            pos = nx.drawing.spring_layout(G)  # Fallback layout
    else:
        pos = nx.drawing.spring_layout(G)  # Default layout if no center name is given
    
    # Extract node labels (e.g., airport names)
    labels = {n: d['name'] for n, d in G.nodes(data=True)}
    
    # Create the plot
    plt.figure(figsize=size)
    nx.draw_networkx(G, pos=pos, node_color=node_color, edge_color='#FFDEA2', with_labels=withLabelsBool, labels=labels, font_size=8)
    plt.show()

# Filter the graph for the node with a specific name and its neighbors
def filter_graph_by_name(G, target_name):
    # Find the node with the specified name
    target_node = None
    for n, d in G.nodes(data=True):
        if d.get('name') == target_name:
            target_node = n
            break
    
    if target_node is None:
        print(f"No node with the name '{target_name}' found.")
        return nx.Graph()  # Return an empty graph if not found
    
    # Get the subgraph induced by the target node and its neighbors
    nodes_to_include = [target_node] + list(G.neighbors(target_node))
    return G.subgraph(nodes_to_include)

# Filter the graph for a specific airport
filtered_graph = filter_graph_by_name(G, "Amsterdam Airport Schiphol")

# Draw the filtered graph with the center node
draw_graph(filtered_graph, size=(50, 50), withLabelsBool=True, center_name="Amsterdam Airport Schiphol")


# Clustering Coefficient
### Menunjukkan bahwa sekitar 1/4 node bersebelahan memiliki tetangga yang juga berhubungan satu sama lain (segitiga) 

In [ ]:
# Calculate the global clustering coefficient
global_clustering = nx.transitivity(G)
print(f"Global Clustering Coefficient: {global_clustering}")

# Calculate the average local clustering coefficient
average_local_clustering = nx.average_clustering(G)
print(f"Average Local Clustering Coefficient: {average_local_clustering}")

# Alternatively, calculate individual clustering coefficients for each node
# node_clustering = nx.clustering(G)
# print("Node Clustering Coefficients:")
# for node, clustering in node_clustering.items():
#     print(f"Node {node} (Airport: {G.nodes[node]['name']}): {clustering}")

# Diameter & Path terpanjang
### Menunjukkan bahwa connecting path paling panjang adalah 10 bandara
### Rata-rata memerlukan 3-4 "trip" untuk sampai ke tujuan akhir

In [ ]:
# Get the largest connected component
largest_cc = max(nx.connected_components(G), key=len)
G_largest_cc = G.subgraph(largest_cc).copy()

# Calculate diameter and average path length for the largest connected component
if nx.is_connected(G_largest_cc):
    diameter = nx.diameter(G_largest_cc)
    avg_path_length = nx.average_shortest_path_length(G_largest_cc)
    print(f"Diameter of the largest connected component: {diameter}")
    print(f"Average Path Length of the largest connected component: {avg_path_length}")
else:
    print("The largest connected component is still disconnected.")


# Scaling Exponent
### Menunjukkan bahwa banyak nodes yang merupakan hubs sedikit dan mayoritas node memiliki degree yang kecil

In [ ]:
import powerlaw
# Get the degree distribution of the graph
degree_sequence = [degree for node, degree in G.degree()]

# Fit the degree distribution to a power law using the powerlaw library
results = powerlaw.Fit(degree_sequence, xmin=1)  # xmin is the minimum degree considered
scaling_exponent = results.alpha  # The scaling exponent (gamma)

# Print the scaling exponent
print(f"Scaling Exponent (alpha): {scaling_exponent}")

# Optional: Plot the degree distribution and the fitted power law
plt.figure(figsize=(10, 6))
plt.hist(degree_sequence, bins=range(1, max(degree_sequence)+1), alpha=0.6, color='b', label='Degree Distribution', density=True)
plt.plot(range(1, max(degree_sequence)+1), results.power_law.pdf(range(1, max(degree_sequence)+1)), color='r', label=f"Fitted Power Law (alpha = {scaling_exponent:.2f})")

plt.xscale('log')  # Set x-axis to logarithmic scale
plt.yscale('log')  # Set y-axis to logarithmic scale
plt.xlabel('Degree')
plt.ylabel('Probability')
plt.legend()
plt.show()

# Assortavity

### Assortivity yang negatif menunjukkan bahwa node yang memiliki degree tinggi berkoneksi ke node yang memiliki degree rendah

Dalam kasus ini, assortivity dekat dengan 0 sehingga kebanyakan airport berhubungan dengan airport lain yang memiliki degree yang sama/lebih banyak 

In [ ]:
# Calculate the assortativity based on degree
assortativity = nx.degree_assortativity_coefficient(G)

print(f"Assortativity coefficient of the graph: {assortativity}")

# Modularity

### Terdapat grup dan komunitas yang sangat "dense" koneksi

In [ ]:
import community  # You may need to install python-louvain package: pip install python-louvain

# Detect communities using the Louvain method
partition = community.best_partition(G)

# Calculate the modularity of the graph given the partition
modularity = community.modularity(partition, G)

print(f"Modularity of the graph: {modularity}")

# Degree Centrality
### 580 adalah airport Amsterdam
Memiliki paling banyak koneksi

In [ ]:
# Calculate degree centrality for the graph G
degree_centrality = nx.degree_centrality(G)

# Sort the degree centrality values to display the top 10 airports with the highest centrality
top_degree_centrality = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

# Prepare results for display
print(top_degree_centrality)

# Average Centrality
average_centrality = sum(degree_centrality.values()) / len(degree_centrality)

print(f"Average centrality: {average_centrality}")


# Betweenness Centrality
### 1382 adalah Charles de Gaulle International Airport di Perancis
Penting menjadi "bridge", mengkoneksikan node paling banyak

In [ ]:
# Calculate betweenness centrality
betweenness_centrality = nx.betweenness_centrality(G, normalized=True)

# Sort the betweenness centrality values to display the top 10 airports with the highest centrality
top_betweenness_centrality = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the results
top_betweenness_centrality

# Closeness Centrality
### 340 adalah Frankfurt am Main Airport di Jerman
"Paling tengah", dekat(sedikit trip) dengan banyak airport

In [ ]:
# Calculate closeness centrality
closeness_centrality = nx.closeness_centrality(G)

# Sort the closeness centrality values to display the top 10 airports with the highest centrality
top_closeness_centrality = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the results
top_closeness_centrality

# EigenVector Centrality
### Amsterdam dan Perancis
Berhubungan dengan airport yang juga merupakan "hub"

In [ ]:
# Calculate eigenvector centrality
eigenvector_centrality = nx.eigenvector_centrality(G)

# Sort the eigenvector centrality values to display the top 10 airports with the highest centrality
top_eigenvector_centrality = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the results
top_eigenvector_centrality

# PageRank Centrality
### 3682 adalah Hartsfield Jackson Atlanta International Airport di US
Jika berhubungan dengan airport populer, nilai PageRank naik

In [ ]:
# Calculate PageRank
pagerank = nx.pagerank(G)

# Sort the PageRank values to display the top 10 airports with the highest centrality
top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:10]

# Display the results
top_pagerank